<p align="center">
  <img src="../logo_ufjf.png" width="200"><br>
  <b>UNIVERSIDADE FEDERAL DE JUIZ DE FORA</b><br>
  Programa de Pós-Graduação em Matemática<br>
  Análise Numérica I<br><br>
  Luís Karlos Mendes de Oliveira<br><br>
  <b>Atividade 01:</b> Método de Euler Explícito (MEE) e Método da Série de Taylor (MST)<br><br>
  <p align="left">Prof.: Sandro Rodrigues Mazorche</p>
  <p align="right">23 de setembro de 2025</p>
</p>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mylib as m
import os
import ipynbname

diretorio_tabela = f"tables/{ipynbname.name()}"
diretorio_imagem = f"img/{ipynbname.name()}"
os.makedirs(diretorio_tabela, exist_ok=True)
os.makedirs(diretorio_imagem, exist_ok=True)


## **Questão 2)**
$ \begin{cases}
  x'(t) = r x(t) \left( 1 - \frac{x(t)}{K} \right) \\
  x(0) = x_0 \\
  t \in [0, T]
\end{cases} $

solução exata: 
$ \frac{K x_0 e^{r t}}{K + x_0 (e^{r t} - 1)} $

### Método de Euler Explícito

In [ ]:
resultados = {}


| **Caso**          | **Parâmetros**                                                                 |
|--------------------|-------------------------------------------------------------------------------|
| **Caso 1** | r = 1, K = 1, x₀ = 0.1, h = [0.5, 1.5, 2.1]                              |
| **Caso 2** | r = 2.5, K = 1, x₀ = 0.1, h = [0.1, 0.5, 0.9]                            |
| **Caso 3** | r = 1, K = 1, x₀ = 1.1, h = [0.5, 1.5, 2.1] |
| **Caso 4** | r = 2.5, K = 1, x₀ = 1.1, h = [0.1, 0.5, 0.8] |

In [ ]:
f = lambda t, x: r * x * (1 - x / K)

# Parâmetros do problema
T = 31.5
r = 2.5
K = 1
x0 = 1.1
dt = .8
nome_arquivo = f"{diretorio_imagem}/euler_caso4.pdf"

condicao_inicial = [0, x0]

print(f'Resolvendo o PVI com dt = {dt} ...')
t_euler, y_euler = m.metodo_euler_explicito(f, condicao_inicial, T, dt)
y_exato = K * x0 * np.exp(r * t_euler) / (K + x0 * (np.exp(r * t_euler) - 1))

dados = pd.DataFrame({
  't': t_euler,
  'x_exato(t)': y_exato,
  'x(t)': y_euler,
  'err_loc': m.erro_local(y_exato ,y_euler),
  'err_rel (%)': m.erro_relativo(y_exato ,y_euler)*100
})

resultados[f'dt={dt}'] = dados

fig, ax = plt.subplots(figsize=(12, 7))
t_curva_exata = np.linspace(0, T, 200)
y_curva_exata = K * x0 * np.exp(r * t_curva_exata) / (K + x0 * (np.exp(r * t_curva_exata) - 1))
ax.plot(t_curva_exata, y_curva_exata, 'k--', label='Solução Exata', linewidth=2)
for nome, df in resultados.items():
    ax.plot(df['t'], df['x(t)'], '-', label=f'{nome}')

#ax.set_title('Comparação do Método de Euler com a Solução Exata', fontsize=16)
ax.set_xlabel('t', fontsize=14)
ax.set_ylabel('x(t)', fontsize=14)
ax.legend(fontsize=14)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(nome_arquivo)
print(f"✅ Gráfico salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


### Método da Série de Taylor

In [ ]:
resultados_mst = {}


In [ ]:
f0 = lambda t, x: r * x * (1 - x / K)
f1 = lambda t, x: r * f0(t, x) * (1 - 2 * x / K)
f2 = lambda t, x: r * f1(t, x) * (1 - 2 * x / K) - (2 * r / K) * f0(t, x)**2
f3 = lambda t, x: r * f2(t, x) * (1 - 2 * x / K) - (6 * r / K) * f0(t, x) * f1(t, x)
f4 = lambda t, x: r * f3(t, x) * (1 - 2 * x / K) - (6 * r / K) * f1(t, x)**2 - (8 * r / K) * f0(t, x) * f2(t, x)

derivadas_calculadas = [f0, f1, f2, f3, f4]

# Parâmetros do problema
T = 50
r = 1
K = 1
x0 = .1
dt = 3.1
ordem = 5
nome_arquivo = f"{diretorio_imagem}/mst_caso1.pdf"
condicao_inicial = [0, x0]

t_out, y_taylor = m.metodo_serie_taylor(derivadas_calculadas[:ordem], condicao_inicial, T, dt)
y_exato = K * x0 * np.exp(r * t_out) / (K + x0 * (np.exp(r * t_out) - 1))
e_rel = m.erro_relativo(y_exato, y_taylor) * 100
e_loc = m.erro_local(y_exato, y_taylor)

dados_mst = pd.DataFrame({
    't': t_out,
    'x_exato(t)': y_exato,
    'x(t)': y_taylor,
    'err_loc': e_loc,
    'err_rel (%)': e_rel
})

resultados_mst[f'dt={dt}'] = dados_mst

fig, ax = plt.subplots(figsize=(12, 7))
t_curva_exata = np.linspace(0, T, 200)
y_curva_exata = K * x0 * np.exp(r * t_curva_exata) / (K + x0 * (np.exp(r * t_curva_exata) - 1))
ax.plot(t_curva_exata, y_curva_exata, 'k-', linewidth=2, label='Solução Exata')
for nome, df in resultados_mst.items():
    ax.plot(df['t'], df['x(t)'], '-', label=f'{nome}')

ax.set_xlabel('t', fontsize=14)
ax.set_ylabel('x(t)', fontsize=14)
ax.legend(fontsize=14)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(nome_arquivo)
print(f"✅ Gráfico salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()
